In [19]:
from Project import collector
from Project.SimulatedAgent.AgentEnums import AgentRole

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

number_of_steps = 500
number_of_simulations = 100
width = 30
height = 30
with_comm = False

In [20]:
%%capture
data = collector.single_run_collector(map_width=width, map_height=height, number_of_simulation_steps=number_of_steps ,with_communication=with_comm, seed=123)
data.tail()

In [21]:
# Plot progress single run
plt = make_subplots(rows=2, cols=2)
d = {}
goals = ["Gold", "Killed_Wumpus", "Map_Progress"]
for goalID in range(len(goals)):
    for agent in range(0, 5):
        df = data[data["Agent"] == agent]
        d["x"] = df["Round"].values
        d["y"] = df[goals[goalID]].values
        txt = [f"{agent}: {df["Role"].values[0]}" for _ in range(0, number_of_steps)]
        plt.add_trace(go.Line(
         d, text= txt, showlegend = False
        ), row=(goalID//2)+1, col=goalID%2+1)
    plt.update_xaxes(title_text="Round", row=(goalID//2)+1, col=goalID%2+1)
    plt.update_yaxes(title_text=goals[goalID], row=(goalID//2)+1, col=goalID%2+1)

plt

C:\Users\Luza3\PycharmProjects\Wumpus\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [25]:

data, ss = collector.multiple_run_result_collector(number_of_simulations , map_width=width, map_height=height, number_of_simulation_steps=number_of_steps ,with_communication=with_comm, start_seed=140)

seed: 140


ValueError: Knowledge base is trying to remove a breeze.

In [23]:
%%capture cap --no-stderr
def compress_data(df: pd.DataFrame, simulation: int):
    tempData = []
    columns=["Role", "Gold", "Killed_Wumpus", "Map_Progress"]
    for i in range(1, simulation):
        for role in AgentRole:
            temp = df.where(df["Role"] == role.name).where(df["Simulation"] == i)
            temp = temp[temp["Agent"].notna()]
            num = len(temp.index)
            row = [0 for _ in range(len(columns))]
            if num != 0:
                tempData.append([role.name, temp.loc[:,"Gold"].mean(), temp.loc[:,"Killed_Wumpus"].mean(), temp.loc[:,"Map_Progress"].mean()])

    df = pd.DataFrame(tempData, columns=columns)
    columns=["Role", "Gold", "Killed_Wumpus", "Map_Progress"]
    tempData = []
    for role in AgentRole:
        tempData.append([role.name, df[df["Role"] == role.name].loc[:,"Gold"].mean(), df[df["Role"] == role.name].loc[:,"Killed_Wumpus"].mean(),df[df["Role"] == role.name].loc[:,"Map_Progress"].mean()])
    return pd.DataFrame(tempData, columns=columns)
        
data = compress_data(data, number_of_simulations)
ss

In [24]:
# Plot progress single run
plt = make_subplots(rows=1, cols=3, horizontal_spacing=0.1)
goals = ["Gold", "Killed_Wumpus", "Map_Progress"]

for goalID in range(len(goals)):
    plt.add_trace(go.Bar(x=data['Role'].to_list(), y=data[goals[goalID]].to_list()
                         , showlegend = False)
                         , row=(goalID//3)+1, col=goalID%3+1)
    plt.update_xaxes(title_text="", row=(goalID//3)+1, col=goalID%3+1)
    plt.update_yaxes(title_text=goals[goalID], row=(goalID//3)+1, col=goalID%3+1)

plt